In [1]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq    

In [2]:
uber = pd.read_parquet('uber_final_uber_transformados_merged.parquet')

In [3]:
yellow = pd.read_parquet('yellow_taxis_final_yellow_taxis_transformado_merged.parquet')

In [4]:
yellow['PUDate'] = pd.to_datetime(yellow['PUDate'])

# Convierte la columna 'PUTime' a datetime
yellow['PUTime'] = pd.to_datetime(yellow['PUTime'], format='%H:%M:%S', errors='coerce')

yellow["PUHour"] = yellow["PUTime"].dt.hour

yellow["PUMonth"] = yellow["PUDate"].dt.month

# Se les otorga una numeración a los dias a fin de traspolarlo luego en una nueva columna
dias_a_numeros = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

yellow['PUDay'] = yellow['PUDate'].dt.day_name().map(dias_a_numeros)

yellow_filtrado = yellow[['PULocationID', 'DOLocationID','PUDate','PUHour','PUMonth','PUDay']]

In [5]:
yellow_filtrado

,PULocationID,DOLocationID,PUDate,PUHour,PUMonth,PUDay
0,249,170,2019-01-23,0,1,3
1,249,170,2019-01-23,9,1,3
2,249,170,2019-01-08,7,1,2
3,249,170,2019-01-05,1,1,6
4,249,170,2019-01-27,0,1,7
...,...,...,...,...,...,...
27501054,207,207,2023-11-30,11,11,4
27501055,207,207,2023-11-20,5,11,1
27501056,51,58,2023-11-29,5,11,3
27501057,156,187,2023-11-20,2,11,1


In [6]:
uber['PUDate'] = pd.to_datetime(uber['PUDate'])

# Convierte la columna 'PUTime' a datetime
uber['PUTime'] = pd.to_datetime(uber['PUTime'], format='%H:%M:%S', errors='coerce')

uber["PUHour"] = uber["PUTime"].dt.hour

uber["PUMonth"] = uber["PUDate"].dt.month

# Se les otorga una numeración a los dias a fin de traspolarlo luego en una nueva columna
dias_a_numeros = {
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
}

uber['PUDay'] = uber['PUDate'].dt.day_name().map(dias_a_numeros)

uber_filtrado = uber[['PULocationID', 'DOLocationID','PUDate','PUHour','PUMonth','PUDay']]

In [7]:
uber_filtrado

,PULocationID,DOLocationID,PUDate,PUHour,PUMonth,PUDay
0,76,35,2023-05-28,23,5,7
1,76,35,2023-05-02,8,5,2
2,76,35,2023-05-06,15,5,6
3,76,35,2023-05-08,16,5,1
4,76,35,2023-05-12,17,5,5
...,...,...,...,...,...,...
55189587,58,253,2023-06-09,20,6,5
55189588,249,105,2023-06-24,21,6,6
55189589,109,110,2023-06-08,14,6,4
55189590,109,110,2023-06-09,14,6,5


In [8]:
uber_filtrado.to_parquet("uber_modelo.parquet",index=False)

In [9]:
yellow_filtrado.to_parquet("yellow_modelo.parquet",index=False)

In [10]:
modelo_ordenado = uber_filtrado.sort_values(by='PUDate').reset_index(drop=True)

In [11]:
modelo_ordenado

,PULocationID,DOLocationID,PUDate,PUHour,PUMonth,PUDay
0,168,127,2019-02-01,7,2,5
1,236,140,2019-02-01,23,2,5
2,255,97,2019-02-01,21,2,5
3,255,97,2019-02-01,12,2,5
4,255,97,2019-02-01,22,2,5
...,...,...,...,...,...,...
55189587,205,10,2023-12-31,9,12,7
55189588,205,10,2023-12-31,23,12,7
55189589,138,79,2023-12-31,14,12,7
55189590,138,265,2023-12-31,16,12,7


In [12]:
#Agrupar el dataset para el modelo
uber_agrupado = modelo_ordenado.groupby(["PULocationID", "PUDay", "PUMonth", "PUHour"])["PULocationID"].count().reset_index(name="Demand")

In [13]:
uber_agrupado

,PULocationID,PUDay,PUMonth,PUHour,Demand
0,1,2,1,6,1
1,1,2,1,16,1
2,1,2,2,8,1
3,1,2,2,23,1
4,1,2,3,18,1
...,...,...,...,...,...
504079,265,7,12,18,2
504080,265,7,12,19,1
504081,265,7,12,20,3
504082,265,7,12,21,3


In [14]:
#Agrupar el dataset para el modelo
yellow_agrupado = yellow_filtrado.groupby(["PULocationID", "PUDay", "PUMonth", "PUHour"])["PULocationID"].count().reset_index(name="Demand")

In [15]:
yellow_agrupado

,PULocationID,PUDay,PUMonth,PUHour,Demand
0,1,1,1,7,2
1,1,1,1,13,4
2,1,1,1,14,2
3,1,1,1,15,2
4,1,1,1,16,6
...,...,...,...,...,...
218912,265,7,12,19,12
218913,265,7,12,20,4
218914,265,7,12,21,4
218915,265,7,12,22,4


In [17]:
#Definir los cuartiles para la demanda

limite_muy_baja = yellow_agrupado['Demand'].quantile(0.20)
limite_baja = yellow_agrupado['Demand'].quantile(0.40)
limite_normal = yellow_agrupado['Demand'].quantile(0.60)
limite_alta = yellow_agrupado['Demand'].quantile(0.80)

In [18]:
#Definir los cuartiles para la demanda

limite_muy_baja = uber_agrupado['Demand'].quantile(0.20)
limite_baja = uber_agrupado['Demand'].quantile(0.40)
limite_normal = uber_agrupado['Demand'].quantile(0.60)
limite_alta = uber_agrupado['Demand'].quantile(0.80)

In [19]:
# Función para categorizar la demanda
def categorizar_demand(demand):
    if demand <= limite_muy_baja:
        return 0
    elif demand <= limite_baja:
        return 1
    elif demand <= limite_normal:
        return 2
    elif demand <= limite_alta:
        return 3
    else:
        return 4

In [20]:
# Aplicar la función de categorización a la columna 'demand'
yellow_agrupado['Demand'] = yellow_agrupado['Demand'].apply(categorizar_demand)



In [24]:
uber_agrupado['Demand'] = uber_agrupado['Demand'].apply(categorizar_demand)

In [30]:
yellow_agrupado

,PULocationID,PUDay,PUMonth,PUHour,Demand
0,1,1,1,7,0
1,1,1,1,13,0
2,1,1,1,14,0
3,1,1,1,15,0
4,1,1,1,16,0
...,...,...,...,...,...
218912,265,7,12,19,0
218913,265,7,12,20,0
218914,265,7,12,21,0
218915,265,7,12,22,0


In [29]:
uber_agrupado

,PULocationID,PUDay,PUMonth,PUHour,Demand
0,1,2,1,6,0
1,1,2,1,16,0
2,1,2,2,8,0
3,1,2,2,23,0
4,1,2,3,18,0
...,...,...,...,...,...
504079,265,7,12,18,0
504080,265,7,12,19,0
504081,265,7,12,20,0
504082,265,7,12,21,0


## REALIZACION DEL CONCAT (LAS OPERACIONES SIGUIENTES SE VUELVEN MUY PESADAS POR LO QUE VOY A DEJAR EL CONCAT PARA LO ULTIMO)

In [26]:
# Se enlistan los dataframes para concatenarlos
archivos = [uber_agrupado, yellow_agrupado]

# Se concatenan los dataframes verticalmente (a lo largo de las filas)
resultado = pd.concat(archivos, ignore_index=True)

# Sonvierte la columna 'PUDate' al formato datetime si aún no lo está
#resultado['PUDate'] = pd.to_datetime(resultado['PUDate'])

# Se organiza el dataframe por la columna 'PUDate'
#modelo = resultado.sort_values(by='PUDate').reset_index(drop=True)


In [28]:
resultado

,PULocationID,PUDay,PUMonth,PUHour,Demand
0,1,2,1,6,0
1,1,2,1,16,0
2,1,2,2,8,0
3,1,2,2,23,0
4,1,2,3,18,0
...,...,...,...,...,...
722996,265,7,12,19,0
722997,265,7,12,20,0
722998,265,7,12,21,0
722999,265,7,12,22,0
